In [11]:
## some options
inputBibFileName = "../../_bibliography/papers.bib"
outputJSONFileName = "collab_net.json"
#
authorInformationFile = (
    ""  #'authorinfoRosalindFranklin.csv' # optional co-author information
)
deleteEgoNode = False
##

import csv  # for loading comma seperated values
import json  # for writing the json
import re

# import necessary libraries
from pybtex.database.input import bibtex  # for reading the bib files


# some auxiliary functions
# you migth have to add further repalcement rules
def latex2unicode(latexString):
    """takes the name of an author as string and return the string
    with latex character replaced as normal string for the HTML"""
    latexString = latexString.replace('{\\"u}', "ü")
    latexString = latexString.replace("{\\'o}", "ó")
    latexString = latexString.replace("{\\'a}", "á")
    latexString = latexString.replace("{\\~a}", "ã")

    return latexString


# some preperation to read the bibtex file
parser = bibtex.Parser()
bib_data = parser.parse_file(inputBibFileName)


listOfAuthors = []  # empty list of authors

In [12]:
# clean data
for paperKeys in bib_data.entries.keys():
    # convert author names to strings and remove * for co-first
    thors = [latex2unicode(str(author)) for author in bib_data.entries[paperKeys].persons["author"] ]
    thors = [author.replace("*", "") for author in thors]
    bib_data.entries[paperKeys].persons["author"] = thors
    # remove latex formatting from titles "{}"
    title = bib_data.entries[paperKeys].fields["title"]
    title = title.replace("{", "")
    title = title.replace("}", "")
    bib_data.entries[paperKeys].fields["title"] = title
    
print(bib_data.entries[paperKeys].persons["author"])

['Duncan, R.', 'Lucas, M.']


In [13]:
listOfAuthors = []  # empty list of authors

# go throuh all entries
for paperKeys in bib_data.entries.keys():
    # get the authors of this paper

    # authors = bib_data.entries[paperKeys].persons['author'].split(" and ")
    # save them to the list of authors
    for author in bib_data.entries[paperKeys].persons["author"]:
        listOfAuthors.append(str(author))

listOfAuthors

# remove * for co-first 
#listOfAuthors = [author.replace("*", "") for author in listOfAuthors]

['Moriamé, M.',
 'Lucas, M.',
 'Carletti, T.',
 'Lucas, M.',
 'Gallo, L.',
 'Ghavasieh, A.',
 'Battiston, F.',
 'De Domenico, M.',
 'Nurisso, M.',
 'Morandini, M.',
 'Lucas, M.',
 'Vaccarino, F.',
 'Gili, T.',
 'Petri, G.',
 'Robiglio, T.',
 'Neri, M.',
 'Coppes, D.',
 'Agostinelli, C.',
 'Battiston, F.',
 'Lucas, M.',
 'Petri, G.',
 'Santoro, A.',
 'Battiston, F.',
 'Lucas, M.',
 'Petri, G.',
 'Amico, E.',
 'Zhang, Y.',
 'Skardal, P. S.',
 'Battiston, F.',
 'Petri, G.',
 'Lucas, M.',
 'Brondetta, A.',
 'Bizyaeva, A.',
 'Lucas, M.',
 'Petri, G.',
 'Musslick, S.',
 'Leitão, A.',
 'Lucas, M.',
 'Poetto, S.',
 'Hersh, T. A.',
 'Gero, S.',
 'Gruber, D.',
 'Bronstein, M.',
 'Petri, G.',
 'Nurisso, M.',
 'Arnaudon, A.',
 'Lucas, M.',
 'Peach, R. L.',
 'Expert, P.',
 'Vaccarino, F.',
 'Petri, G.',
 'Lucas, M.',
 'Townsend-Teague, A.',
 'Neri, M.',
 'Poetto, S.',
 'Morris, A.',
 'Habermann, B. H.',
 'Tichit, L.',
 'Landry, N. W.',
 'Lucas, M.',
 'Iacopini, I.',
 'Petri, G.',
 'Schwarze, A.',
 

In [14]:


if deleteEgoNode == True:
    # we assume that the author with the most entries is the ego node and delete it
    egoNode = max(listOfAuthors, key=listOfAuthors.count)  # returns the ego node
    listOfAuthors = list(set(listOfAuthors))  # gets unique list of authors
    listOfAuthors.pop(
        listOfAuthors.index(egoNode)
    )  # deletes it from the list of authors
    print("Removing the ego node: %s " % egoNode)
else:
    listOfAuthors = list(set(listOfAuthors))  # gets unique list of authors

nAuthors = len(listOfAuthors)  # number of author nodes
nPapers = len(bib_data.entries.keys())  # number of paper nodes


# read the additional author information from the csv
authorLinks_dict = {}  # create an empty dictionary
authorImage_dict = {}  # create an empty dictionary

try:
    authorInfo_reader = csv.DictReader(open(authorInformationFile))
    for row in authorInfo_reader:
        authorLinks_dict[row["name"]] = row["url"]
        authorImage_dict[row["name"]] = row["image"]
except FileNotFoundError:
    print("no optional co-author information available")

no optional co-author information available


In [15]:
# create a dictionary reflecting the graph (there are more pythonic ways
# possible to creat this, e.g., with zip, but this is easiest)

node_list = []
# create author nodes
for i in range(nAuthors):
    node_dict = {}  # create an empty dictionary for this node
    node_dict["id"] = "A" + str(i)
    node_dict["group"] = 0
    # invert the name such that the given name is before the last name
    try:  # we need this try to deal with single author papers
        authorSplit = listOfAuthors[i].split(",")
        nameThisAuthor = authorSplit[1][1::] + " " + authorSplit[0]
    except:
        authorSplit = listOfAuthors[i]
        nameThisAuthor = authorSplit[1][1::] + " " + authorSplit[0]

    nameThisAuthorUnicode = latex2unicode(nameThisAuthor)
    print(nameThisAuthorUnicode)
    node_dict["name"] = nameThisAuthorUnicode
    node_list.append(node_dict)
    # try to set the url for this author but default is google it
    node_dict["url"] = "https://www.google.com/search?q=" + nameThisAuthor
    try:
        if authorLinks_dict[nameThisAuthor] is None:
            raise KeyError("no information for this author")
        else:
            node_dict["url"] = authorLinks_dict[nameThisAuthor]
    except KeyError:
        node_dict["url"] = "https://www.google.com/search?q=" + nameThisAuthor

        # try to set a image for this author
    try:
        node_dict["image"] = authorImage_dict[nameThisAuthor]
    except KeyError:  # if no image jsut leave blank
        node_dict["image"] = []

L. Gallo
A. Ghavasieh
A. Brondetta
M. Bronstein
A. Bizyaeva
I. Iacopini
T. Robiglio
J. Petit
L. Torres
R. L. Peach
P. S. Skardal
T. A. Hersh
J. Newman
A. Leitão
D. Gruber
P. Expert
D. Coppes
A. Morris
B. H. Habermann
T. Gili
M. Neri
V. Latora
Y. Zhang
S. Musslick
D. Fanelli
T. Carletti
A. Townsend-Teague
F. Battiston
A. Santoro
E. Amico
M. Morandini
A. Arnaudon
A. Patania
L. Tichit
A. Stefanovska
M. Lucas
J.-G. Young
R. Duncan
C. Agostinelli
A. Schwarze
M. De Domenico
S. Poetto
S. Gero
G. Cencetti
M. Nurisso
F. Vaccarino
N. W. Landry
M. Moriamé
G. Petri
A. Barrat


In [17]:
# create the links between the nodes
link_list = []
i=0
for paperKeys in bib_data.entries.keys(): # go over every paper
    # create the paper node
    node_dict = {} # create an empty dictionary for this node
    node_dict["id"] = "P" + str(i)
    node_dict["group"] = 1
    thisPaperName = bib_data.entries[paperKeys].fields['title']
    # remove curly bracket in paper name, remove it
    if thisPaperName[0]=='{':
        thisPaperName=thisPaperName[1:-1]
    node_dict["name"] =  thisPaperName
    node_list.append(node_dict)

    # set image for this paper
    try:
        node_dict["image"] = bib_data.entries[paperKeys].fields['image']
    except KeyError: # if no image just leave blank
        node_dict["image"] = []


    # find the authors for this paper
    authorsThisPaper = [str(i).split('*')[0].strip() for i in bib_data.entries[paperKeys].persons['author']]

    # if the paper has a url, add it
    try:
        node_dict["url"] = bib_data.entries[paperKeys].fields['url']
    except KeyError: # otherwise refer to google
        node_dict["url"] = "https://www.google.com/search?q=" + bib_data.entries[paperKeys].fields['title']


    for authors in authorsThisPaper:
        link_dict = {} # empty dictionary for this edge
        link_dict["source"] = "P" + str(i) # attached to this paper
        try:
            link_dict["target"] = "A" + str(listOfAuthors.index(authors)) # and attached to co-author
            link_list.append(link_dict)    # save it into the list
        except ValueError:
            pass
            #print("Author %s not in list, probably the ego node." %authors )
    i=i+1

# write into dictionary
graph_dict = {"nodes" : node_list, "links" : link_list}


# opening the file to write
if outputJSONFileName:
    # Writing JSON data
    with open(outputJSONFileName, 'w') as f:
        json.dump(graph_dict, f, indent=4)


In [35]:
for i, node in enumerate(node_list):
    if node["name"] == "M. Lucas":
        idx = node["id"]
        ii = i
        print(node)
        break
    else: 
        ii=None
        
        
        

In [33]:
# remove myseflnode_list.pop(ii)

{'id': 'A35',
 'group': 0,
 'name': 'M. Lucas',
 'url': 'https://www.google.com/search?q=M. Lucas',
 'image': []}

In [ ]:
[i for i in link_list if idx not in i.values()]

In [34]:
node_list

[{'id': 'A0',
  'group': 0,
  'name': 'L. Gallo',
  'url': 'https://www.google.com/search?q=L. Gallo',
  'image': []},
 {'id': 'A1',
  'group': 0,
  'name': 'A. Ghavasieh',
  'url': 'https://www.google.com/search?q=A. Ghavasieh',
  'image': []},
 {'id': 'A2',
  'group': 0,
  'name': 'A. Brondetta',
  'url': 'https://www.google.com/search?q=A. Brondetta',
  'image': []},
 {'id': 'A3',
  'group': 0,
  'name': 'M. Bronstein',
  'url': 'https://www.google.com/search?q=M. Bronstein',
  'image': []},
 {'id': 'A4',
  'group': 0,
  'name': 'A. Bizyaeva',
  'url': 'https://www.google.com/search?q=A. Bizyaeva',
  'image': []},
 {'id': 'A5',
  'group': 0,
  'name': 'I. Iacopini',
  'url': 'https://www.google.com/search?q=I. Iacopini',
  'image': []},
 {'id': 'A6',
  'group': 0,
  'name': 'T. Robiglio',
  'url': 'https://www.google.com/search?q=T. Robiglio',
  'image': []},
 {'id': 'A7',
  'group': 0,
  'name': 'J. Petit',
  'url': 'https://www.google.com/search?q=J. Petit',
  'image': []},
 {'id'

In [30]:
[1, 3, 4, 10].pop(1)

3

In [ ]:
link

[{'source': 'P0', 'target': 'A47'},
 {'source': 'P0', 'target': 'A25'},
 {'source': 'P1', 'target': 'A0'},
 {'source': 'P1', 'target': 'A1'},
 {'source': 'P1', 'target': 'A27'},
 {'source': 'P1', 'target': 'A40'},
 {'source': 'P2', 'target': 'A44'},
 {'source': 'P2', 'target': 'A30'},
 {'source': 'P2', 'target': 'A45'},
 {'source': 'P2', 'target': 'A19'},
 {'source': 'P2', 'target': 'A48'},
 {'source': 'P3', 'target': 'A6'},
 {'source': 'P3', 'target': 'A20'},
 {'source': 'P3', 'target': 'A16'},
 {'source': 'P3', 'target': 'A38'},
 {'source': 'P3', 'target': 'A27'},
 {'source': 'P3', 'target': 'A48'},
 {'source': 'P4', 'target': 'A28'},
 {'source': 'P4', 'target': 'A27'},
 {'source': 'P4', 'target': 'A48'},
 {'source': 'P4', 'target': 'A29'},
 {'source': 'P5', 'target': 'A22'},
 {'source': 'P5', 'target': 'A10'},
 {'source': 'P5', 'target': 'A27'},
 {'source': 'P5', 'target': 'A48'},
 {'source': 'P6', 'target': 'A2'},
 {'source': 'P6', 'target': 'A4'},
 {'source': 'P6', 'target': 'A48'

In [44]:
i

102

In [10]:
node_list

[{'id': 'A0',
  'group': 0,
  'name': 'L. Gallo',
  'url': 'https://www.google.com/search?q=L. Gallo',
  'image': []},
 {'id': 'A1',
  'group': 0,
  'name': 'A. Ghavasieh',
  'url': 'https://www.google.com/search?q=A. Ghavasieh',
  'image': []},
 {'id': 'A2',
  'group': 0,
  'name': 'A. Brondetta',
  'url': 'https://www.google.com/search?q=A. Brondetta',
  'image': []},
 {'id': 'A3',
  'group': 0,
  'name': 'M. Bronstein',
  'url': 'https://www.google.com/search?q=M. Bronstein',
  'image': []},
 {'id': 'A4',
  'group': 0,
  'name': 'A. Bizyaeva',
  'url': 'https://www.google.com/search?q=A. Bizyaeva',
  'image': []},
 {'id': 'A5',
  'group': 0,
  'name': 'I. Iacopini',
  'url': 'https://www.google.com/search?q=I. Iacopini',
  'image': []},
 {'id': 'A6',
  'group': 0,
  'name': 'T. Robiglio',
  'url': 'https://www.google.com/search?q=T. Robiglio',
  'image': []},
 {'id': 'A7',
  'group': 0,
  'name': 'J. Petit',
  'url': 'https://www.google.com/search?q=J. Petit',
  'image': []},
 {'id'